# nmi | spring 2024
## homework 04 | systems of equations : iterative
## your name here


### q1 <b><font color=green>MATH 385 ONLY</font></b> systems of equations, iterative.

<b><font color=red>this is a code question.</font></b>
</br></br>

solve this system of equations
</br></br>

\begin{align}
  3x + y + z &= 6 \\
  x + 3y + z &= 3 \\
  x + y + 3z &= 5.
\end{align}
</br>

using initial guess $x_0 = [0,0,0]$, error $\epsilon = abs(||x_{i+1}||_2 - ||x_i||_2) < 1e-5$ and <b>one</b> of the iterative methods below. provide final solution and number of iterations required to meet the error condition.
</br>


#### code, common to (a),(b),(c)


In [ ]:
# used by all (a),(b),(c)

import numpy as np

#https://numpy.org/doc/stable/reference/generated/numpy.dot.html

a = np.array([[3.,1.,1.],[1.,3.,1.],[1.,1.,3.]])
b = np.array([[6.],[3.],[5.]]) # row vector vs row

# system
x_sys = np.linalg.solve(a,b) # system to check your work
print(f"system solution: {x_sys.flatten()}")

x0 = np.array([[0.],[0.],[0.]]) # first guess
eps = 1e-5

d = np.zeros_like(a)
np.fill_diagonal(d,np.diagonal(a))
inv_d = np.linalg.inv(d)
l = np.tril(a,k=-1) # k = -1 ~ below diagonal
u = np.triu(a,k=1)  # k = +1 ~ above diagonal


system solution: [1.6 0.1 1.1]


#### a) (20 pts) jacobi method;
</br>


\begin{align}
  x_0 &= \text{initial vector} \\
  x_{k+1} &= D^{-1}(b-(L+U)x_k) \quad k = 0,1,2,\dots
\end{align}
</br>


In [ ]:
# jacobi

x_old = x0.copy()
x_new = x_old.copy()
error = 2*eps # ensures while-loop runs at least once
count = 0

while error > eps:
  x_new = np.dot(inv_d,b - np.dot(l+u,x_old))
  error = abs(np.linalg.norm(x_new) - np.linalg.norm(x_old))
  x_old = x_new.copy()
  count += 1
  print(f"jac step {count} ",x_new.flatten(),error)
print() # spacer


jac step 1  [2.         1.         1.66666667] 2.788866755113585
jac step 2  [ 1.11111111 -0.22222222  0.66666667] 1.474182358869226
jac step 3  [1.85185185 0.40740741 1.37037037] 1.0248136329932345
jac step 4  [ 1.40740741 -0.07407407  0.91358025] 0.6599411095632857
jac step 5  [1.72016461 0.22633745 1.22222222] 0.4427112262852224
jac step 6  [1.51714678 0.01920439 1.01783265] 0.295225057256302
jac step 7  [1.65432099 0.15500686 1.15454961] 0.19626851890584596
jac step 8  [1.56348118 0.0637098  1.06355738] 0.13130562489717823
jac step 9  [1.62424427 0.12432048 1.12426967] 0.08728867358781489
jac step 10  [1.58380328 0.08382868 1.08381175] 0.05832927582024339
jac step 11  [1.61078652 0.11079499 1.11078934] 0.03882076005856039
jac step 12  [1.59280522 0.09280804 1.09280616] 0.025912536071556413
jac step 13  [1.60479526 0.10479621 1.10479558] 0.0172602590410027
jac step 14  [1.59680274 0.09680305 1.09680284] 0.011513721274869937
jac step 15  [1.60213137 0.10213147 1.1021314 ] 0.007672693

#### b) (30 pts) gauss-seidel; or
</br>


\begin{align}
  x_0 &= \text{initial vector} \\
  x_{k+1} &= D^{-1}(b-Ux_k -Lx_{k+1}) \quad k=0,1,2,\dots
\end{align}
</br>


In [ ]:
# gauss-seidel

x_old = x0.copy()
x_new = x_old.copy()
error = 2*eps # ensures while-loop runs at least once
count = 0

n = len(x_old)
while error > eps:
  for k in range(n): # per row k
    x_new[k] = inv_d[k,k]*(b[k] - np.dot(u[k],x_old) - np.dot(l[k],x_new))
  error = abs(np.linalg.norm(x_new) - np.linalg.norm(x_old))
  x_old = x_new.copy()
  count += 1
print(f"gas step {count} ",x_old.flatten(),error)
print() # spacer


gas step 8  [1.60000578 0.09999728 1.09999898] 6.660436568273198e-06



#### c) (40 pts) successive over-relaxation. include optimal weight $\omega$ as part of solution.
</br>


\begin{align}
  x_0 &= \text{initial vector} \\
  x_{k+1} &= (1-\omega)x_k + \omega(D^{-1}(b-Ux_k - Lx_{k+1})) \quad k=0,1,2,\dots
\end{align}
</br>


In [ ]:
# sor

import matplotlib.pyplot as plt
from tabulate import tabulate

n_act = np.linalg.norm(x_sys)
n = len(x0)
nws = 25 + 1
ws = np.linspace(1,1.25,nws)
data = np.zeros((nws,5+n)) # wi,w,x,count,error,Δ norms

# loop to find best w
for wi,w in enumerate(ws):
  inv_dwl = np.linalg.inv(d + w*l)
  x_old = x0.copy()
  x_new = x_old.copy()
  error = 2*eps # ensures while-loop runs at least once
  count = 0

  while error > eps:
    for k in range(n): # per row k
      x_new[k] = (1-w)*x_old[k] + w*(inv_d[k,k]*(b[k] - np.dot(u[k],x_old) - np.dot(l[k],x_new)))
    error = abs(np.linalg.norm(x_new) - np.linalg.norm(x_old))
    x_old = x_new.copy()
    count += 1

  data[wi,0] = wi+1
  data[wi,1] = w
  for k in range(n):
    data[wi,2+k] = x_new[k][0]
  data[wi,3+k] = count
  data[wi,4+k] = error
  data[wi,5+k] = abs(np.linalg.norm(x_new) - n_act)
  #print(f"w = {w}, sor step {count} ",x_old.flatten(),error)
print() # spacer
print(tabulate(data,headers=['i',chr(969),"x1","x2","x3","count","error",chr(916)+" L2-norm"],tablefmt="github"),"\n")



|   i |    ω |      x1 |        x2 |   x3 |   count |       error |   Δ L2-norm |
|-----|------|---------|-----------|------|---------|-------------|-------------|
|   1 | 1    | 1.60001 | 0.0999973 |  1.1 |       8 | 6.66044e-06 | 4.0389e-06  |
|   2 | 1.01 | 1.60002 | 0.0999851 |  1.1 |       7 | 5.64262e-06 | 1.44329e-05 |
|   3 | 1.02 | 1.6     | 0.1       |  1.1 |       9 | 2.62337e-06 | 1.4739e-07  |
|   4 | 1.03 | 1.6     | 0.1       |  1.1 |       9 | 1.78343e-06 | 1.46323e-07 |
|   5 | 1.04 | 1.6     | 0.1       |  1.1 |       9 | 7.1614e-07  | 3.69035e-07 |
|   6 | 1.05 | 1.6     | 0.1       |  1.1 |       9 | 4.75973e-07 | 4.93475e-07 |
|   7 | 1.06 | 1.6     | 0.100002  |  1.1 |       8 | 6.89809e-06 | 2.18481e-06 |
|   8 | 1.07 | 1.6     | 0.100001  |  1.1 |       8 | 1.64846e-06 | 3.17338e-06 |
|   9 | 1.08 | 1.59999 | 0.0999999 |  1.1 |       8 | 4.84995e-06 | 3.81191e-06 |
|  10 | 1.09 | 1.6     | 0.0999992 |  1.1 |       9 | 4.22565e-06 | 2.45639e-07 |
|  11 | 1.1  | 


### q2) (5 pts) discuss all three methods of q1.

### q3) (10 pts) <b><font color=orange>extra credit</font></b>. tiny linear algebra proof = free points!


let $A\in M_n(\mathbb{R})$. we say that $A$ is an <i>orthogonal</i> matrix if the columns of $A$ are mutually orthogonal. specifically, if $A = [a_1,a_2,\dots,a_n]$, where $a_j$ denotes the columns of $A$, then $a_i\cdot a_j = 0$, whenever $i\ne j$, where $v\cdot w$ denotes the euclidean dot product -- ie, $v\cdot w = \sum_{i=1}^n v_i,w_i$. prove that if $A$ is an orthogonal matrix, then $A^TA$ is a diagonal matrix, where $A^T$ denotes the transpose of $A$.

<b>note: every orthonormal set is also orthogonal, but not every orthogonal set is orthonormal.</b>
</br>


### q4) (10 pts) cholesky factorization.

solve the system by finding the cholesky factorization of $A_{2\times 2}$ followed by two back substitutions. show your work.
</br></br>

$$
\begin{bmatrix} 1 & -1 \\ -1 & 5 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
=
\begin{bmatrix} 3 \\ -7 \end{bmatrix}
$$
</br>


### q5) (10 pts) conjugate gradient method.

solve the system with conjugate gradient method. show your work.

<b><font color=red>the replacement:</font></b>
</br></br>

$$
\begin{bmatrix} 1 & 2 \\ 2 & 5 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
=
\begin{bmatrix} 1 \\ 1 \end{bmatrix}
$$
</br>

<b><font color=red>what it replaced:</font></b>
</br></br>
$$
\begin{bmatrix} 1 & -1 \\ -1 & 5 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
=
\begin{bmatrix} 3 \\ -7 \end{bmatrix}
$$
</br>


##### answer


###### code


In [ ]:
# nmi, hw04, q5 with sympy to display unevaluated fractions

import numpy as np
import sympy as s

def sym_cg(aA,ab,ax0,adisplay=True):
  A,x_old,b,r_old,d,α,β = s.symbols("A,x_old,b,r_old,d,α,β")

  A = s.Matrix(aA)
  b = s.Matrix(ab)
  x_old = s.Matrix(ax0)
  x_new = None

  Ax = s.Matrix([A.row(0).dot(x_old),A.row(1).dot(x_old)])
  r_old = b.copy() - Ax
  d_old = r_old.copy()

  r_zero = s.Matrix([[0],[0]])
  n = len(x_old)
  for i in range(n):
    if r_old == r_zero:
      break

    Ad = s.Matrix([A.row(0).dot(d_old),A.row(1).dot(d_old)])
    α = r_old.col(0).dot(r_old) / d_old.dot(Ad)
    x_new = x_old + α*d_old
    r_new = r_old - α*Ad
    β = r_new.col(0).dot(r_new) / r_old.col(0).dot(r_old)
    d_new = r_new + β*d_old

    if adisplay:
      print(f"\nstep : {i+1} of {n}\n")
      print(f"A·d[{i}] : {Ad}")
      print(f"α : {α}")
      print(f"x[{i+1}] : {x_new}")
      print(f"r[{i+1}] : {r_new}")
      print(f"β : {β}")
      print(f"d[{i+1}] : {d_new}")

    x_old = x_new.copy()
    r_old = r_new.copy()
    d_old = d_new.copy()

  return x_new

def main():
  # problem, original
  A = [[[1,-1],[-1,5]]] # 3rd set nested brackets for subsequent problems
  b = [[[3],[-7]]]
  x0 = [[[0],[0]]]

  # problem w smaller RHS
  A.append([[1,2],[2,5]])
  b.append([[1],[1]])
  x0.append([[0],[0]])
  print(A[0])

  for i in range(len(A)):
    print(f"\nversion {i}, solution x : {sym_cg(A[i],b[i],x0[i])}")

if __name__ == "__main__":
  main()


[[1, -1], [-1, 5]]

step : 1 of 2

A·d[0] : Matrix([[10], [-38]])
α : 29/148
x[1] : Matrix([[87/148], [-203/148]])
r[1] : Matrix([[77/74], [33/74]])
β : 121/5476
d[1] : Matrix([[6061/5476], [1595/5476]])

step : 2 of 2

A·d[1] : Matrix([[2233/2738], [957/2738]])
α : 37/29
x[2] : Matrix([[2], [-1]])
r[2] : Matrix([[0], [0]])
β : 0
d[2] : Matrix([[0], [0]])

version 0, solution x : Matrix([[2], [-1]])

step : 1 of 2

A·d[0] : Matrix([[3], [7]])
α : 1/5
x[1] : Matrix([[1/5], [1/5]])
r[1] : Matrix([[2/5], [-2/5]])
β : 4/25
d[1] : Matrix([[14/25], [-6/25]])

step : 2 of 2

A·d[1] : Matrix([[2/25], [-2/25]])
α : 5
x[2] : Matrix([[3], [-1]])
r[2] : Matrix([[0], [0]])
β : 0
d[2] : Matrix([[0], [0]])

version 1, solution x : Matrix([[3], [-1]])


### q6) (10 pts) <b><font color=blue>MATH 685 ONLY</font></b> prove that if $d>4$, matrix $A$ is positive-definite.

$$
\begin{bmatrix} 1 & 2 \\ 2 & 5 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
=
\begin{bmatrix} 1 \\ 1 \end{bmatrix}
$$
</br>


### q7) (30 pts) <b><font color=blue>MATH 685 ONLY</font></b> nonlinear systems of equations, newtons multivariate.

<b><font color=red>this is a code question.</font></b>
</br></br>

use newtons method with appropriate $x_0$ to find all solutions. use error $\epsilon = abs(||x^*||_2 -||x||_2) < 1e-05$, where $x^*$ is the true solution. display final solutions, iteration count to get there.
</br></br>

\begin{align}
  x_1^2 + x_2^2 &= 1 \\
  (x_1-1)^2 + x_2^2 &= 1
\end{align}
</br>


\begin{align}
  x_0 &= \text{initial vector} \\
  x_{k+1} &= x_k - (DF(x_k))^{-1}F(x_k), \quad k=0,1,2,\dots
\end{align}
</br>


In [ ]:
import numpy as np

#https://numpy.org/doc/stable/reference/generated/numpy.dot.html

f1 = lambda x: pow(x[0],2) + pow(x[1],2) - 1
f2 = lambda x: pow(x[0]-1,2) + pow(x[1],2) - 1
df11 = lambda x: 2*x[0]
df12 = lambda x: 2*x[1]
df21 = lambda x: 2*x[0] - 2
df22 = lambda x: 2*x[1]

x0 = np.array([1,-1]) # first guess ~ remember what the equations represent graphically
eps = 1e-5
imax = 10 # lol, not even this

x_old = x0.copy()
x_new = x_old.copy()

count = 0
error = 2*eps
while (error > eps) and (count <= imax):
  F = [f1(x_old),f2(x_old)]
  DF = [[df11(x_old),df12(x_old)],[df21(x_old),df22(x_old)]]
  x_new = x_old - np.dot(np.linalg.inv(DF),F)
  error = abs(np.linalg.norm(x_new)-np.linalg.norm(x_old))
  x_old = x_new.copy()
  count += 1

print("\n",x_new.flatten(),error,count)



 [ 0.5       -0.8660254] 1.05908426473178e-09 5
